# Creating Logistic Regression to predict Absenteeism

In [1]:
# importing the relevant libraries
import numpy as np
import pandas as pd

In [2]:
# get the preprocessed data
data_preprocessed = pd.read_csv('Absenteeism_preprocessed.csv')

In [3]:
# check the top 5 values
data_preprocessed.head()

,Reason_1,Reason_2,Reason_3,Reason_4,Month_Value,Day of the Week,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours
0,0,0,0,1,7,1,289,36,33,239.554,30,0,2,1,4
1,0,0,0,0,7,1,118,13,50,239.554,31,0,1,0,0
2,0,0,0,1,7,2,179,51,38,239.554,31,0,0,0,2
3,1,0,0,0,7,3,279,5,39,239.554,24,0,2,0,4
4,0,0,0,1,7,3,289,36,33,239.554,30,0,2,1,2


### Create Targets

As we are doing the logistic regression that means targets can be classified into classes.So here we may form classes like 
employees those are "excessively absent" & the other is "moderately absent". For this we may use median of Absenteeism
time.Employees which are absent greater than the median value are excessively absent and which are less than median are 
moderately absent.

In [4]:
# getting the median of absence time 
absentee_time_median = data_preprocessed['Absenteeism Time in Hours'].median()

In [5]:
# so conversion of time to 1 and 0 can be done with several ways like using apply,map as follows
data_preprocessed['Absenteeism Time in Hours'].apply(lambda x:1 if x>absentee_time_median else 0)

0      1
1      0
2      0
3      1
4      0
      ..
695    1
696    0
697    1
698    0
699    0
Name: Absenteeism Time in Hours, Length: 700, dtype: int64

In [6]:
data_preprocessed['Absenteeism Time in Hours'].map(lambda x:1 if x>absentee_time_median else 0)

0      1
1      0
2      0
3      1
4      0
      ..
695    1
696    0
697    1
698    0
699    0
Name: Absenteeism Time in Hours, Length: 700, dtype: int64

In [7]:
data_preprocessed['Absenteeism Time in Hours'].apply(lambda x:1 if x>absentee_time_median else 0).unique()

array([1, 0], dtype=int64)

In [8]:
# other method we can use is from numpy i.e. where 
targets = np.where(data_preprocessed['Absenteeism Time in Hours']>absentee_time_median,1,0)

In [9]:
# make a new column for encoded targets 
data_preprocessed["Excessive Absenteeism"] = targets

In [10]:
data_preprocessed.head()

,Reason_1,Reason_2,Reason_3,Reason_4,Month_Value,Day of the Week,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours,Excessive Absenteeism
0,0,0,0,1,7,1,289,36,33,239.554,30,0,2,1,4,1
1,0,0,0,0,7,1,118,13,50,239.554,31,0,1,0,0,0
2,0,0,0,1,7,2,179,51,38,239.554,31,0,0,0,2,0
3,1,0,0,0,7,3,279,5,39,239.554,24,0,2,0,4,1
4,0,0,0,1,7,3,289,36,33,239.554,30,0,2,1,2,0


### Small Analysis of targets

In [11]:
# sum of all targets means sumof all 1 divided by the number of observations that will give the average of ones..how 1 is ditributed.
targets.sum()/targets.shape[0]

0.45571428571428574

The  above can also be checked with mean() method of pandas. SO it is indicating that aroud 45% targets are 1 and 55% are 0s.In logistic regression our targets classes should be distributed equally or 60-40% will also work..Our distribution is in line with the condition.

In [12]:

data_preprocessed['Excessive Absenteeism'].mean()

0.45571428571428574

### Drop column 'Absenteeism Time in Hours'

In [13]:
data_with_targets = data_preprocessed.drop(['Absenteeism Time in Hours'],axis = 1)

In [14]:
data_with_targets.head(2)

,Reason_1,Reason_2,Reason_3,Reason_4,Month_Value,Day of the Week,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Excessive Absenteeism
0,0,0,0,1,7,1,289,36,33,239.554,30,0,2,1,1
1,0,0,0,0,7,1,118,13,50,239.554,31,0,1,0,0


### Select inputs for the logistic regression

In [15]:
data_with_targets.shape

(700, 15)

In [16]:
# we are taking all the features as inputs, except last one excessive absenteeism which is a target
unscaled_inputs = data_with_targets.iloc[ : ,:-1]
unscaled_inputs.head(2)

,Reason_1,Reason_2,Reason_3,Reason_4,Month_Value,Day of the Week,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets
0,0,0,0,1,7,1,289,36,33,239.554,30,0,2,1
1,0,0,0,0,7,1,118,13,50,239.554,31,0,1,0


## standardize the data

We are going to standardize our data with sklearn.But the features for which we made dummies that should not be standardised. Otherwise the purpose for which we made dummies that will not be fullfilled. So we avoid standardising the feature with dummies.

So while standardizing we will select the only those features without dummies.

In [17]:
# importing the necessary class from sklearn
from sklearn.preprocessing import StandardScaler

In [18]:
absenteeism_scaler = StandardScaler()

In [19]:
unscaled_inputs.iloc[:,[4,5,6,7,8,9,10,12,13]]= absenteeism_scaler.fit_transform(unscaled_inputs.iloc[:,[4,5,6,7,8,9,10,12,13]])

In [20]:
inputs = unscaled_inputs
inputs.head()

,Reason_1,Reason_2,Reason_3,Reason_4,Month_Value,Day of the Week,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets
0,0,0,0,1,0.182726,-0.683704,1.005844,0.412816,-0.536062,-0.806331,0.767431,0,0.880469,0.268487
1,0,0,0,0,0.182726,-0.683704,-1.574681,-1.141882,2.130803,-0.806331,1.002633,0,-0.019280,-0.589690
2,0,0,0,1,0.182726,-0.007725,-0.654143,1.426749,0.248310,-0.806331,1.002633,0,-0.919030,-0.589690
3,1,0,0,0,0.182726,0.668253,0.854936,-1.682647,0.405184,-0.806331,-0.643782,0,0.880469,-0.589690
4,0,0,0,1,0.182726,0.668253,1.005844,0.412816,-0.536062,-0.806331,0.767431,0,0.880469,0.268487


In [21]:
inputs.shape

(700, 14)

### Split the data into train & test and shuffle

In [22]:
from sklearn.model_selection import train_test_split

In [23]:
train_test_split(inputs,targets)

[     Reason_1  Reason_2  Reason_3  Reason_4  Month_Value  Day of the Week  \
 645         0         0         0         1    -0.959313         1.344231   
 498         0         0         0         1     0.753746        -0.683704   
 143         0         0         0         1    -1.244823         1.344231   
 472         0         0         0         1     0.182726        -0.683704   
 96          1         0         0         0     1.324766        -0.007725   
 ..        ...       ...       ...       ...          ...              ...   
 279         1         0         0         0     0.753746        -0.007725   
 9           0         0         0         1     0.182726        -1.359682   
 169         1         0         0         0    -0.959313         0.668253   
 425         1         0         0         0    -0.388293        -1.359682   
 458         1         0         0         0    -0.102784        -0.683704   
 
      Transportation Expense  Distance to Work       Age  \
 6

In [24]:
x_train,x_test,y_train,y_test = train_test_split(inputs,targets)

In [25]:
print(x_train.shape,y_train.shape)

(525, 14) (525,)


The above shape shows that training inputs have 525 obeservations across 14 features and targets for training is a vector of length 525.

In [26]:
print(x_test.shape,y_test.shape)

(175, 14) (175,)


So here sklearn splits the data as 75% training and 25% testing

#### Generally a split of 80-20 or 90-10 is used. So we will do here a 80%-20% splitting

In [27]:
x_train,x_test,y_train,y_test = train_test_split(inputs,targets,train_size=0.8,random_state=20)

In [28]:
print(x_train.shape,y_train.shape)

(560, 14) (560,)


In [29]:
print(x_test.shape,y_test.shape)

(140, 14) (140,)


## Logistic Regression with Sklearn

In [30]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

### Training the model

In [31]:
reg = LogisticRegression()

In [32]:
reg.fit(x_train,y_train)

LogisticRegression()

#### Check the accuracy of model

In [33]:
reg.score(x_train,y_train)

0.775

### Checking the accuracy manuually

In [34]:
# for cheking the model o/p
model_outputs = reg.predict(x_train)

In [35]:
model_outputs

array([0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0,
       0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0,
       1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1,
       1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1,
       0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0,
       0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0,

In [36]:
y_train

array([0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0,
       1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1,
       1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1,
       1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1,
       0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1,
       1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0,

In [37]:
model_outputs == y_train

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True, False,  True, False, False,  True,  True,  True,  True,
       False,  True, False,  True, False, False,  True,  True,  True,
       False,  True,  True,  True,  True,  True,  True,  True,  True,
        True, False, False, False,  True, False,  True,  True,  True,
        True,  True,  True,  True,  True, False,  True,  True,  True,
        True,  True,  True,  True,  True, False,  True,  True,  True,
        True,  True,  True,  True, False,  True, False,  True,  True,
        True,  True,  True, False,  True,  True,  True,  True,  True,
       False,  True, False,  True,  True, False, False, False,  True,
        True,  True,  True,  True,  True,  True,  True, False,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True, False,  True,  True,  True,  True,
       False,  True,  True,  True,  True, False,  True,  True,  True,
        True,  True,

In [38]:
np.sum(model_outputs==y_train)

434

In [39]:
observations = model_outputs.shape[0]
observations

560

In [40]:
np.sum(model_outputs==y_train)/observations

0.775

So in total of 560 observations our model predicted accurately(prediction= target) for 439 observations.Hence we got the same number. nearly 80 % is accuracy.

### Finding the intercepts & coefficients

In [41]:
reg.intercept_

array([-1.6561092])

In [42]:
reg.coef_

array([[ 2.80096498e+00,  9.34857518e-01,  3.09561645e+00,
         8.56587468e-01,  1.66248119e-01, -8.43703301e-02,
         6.12732578e-01, -7.79685996e-03, -1.65922708e-01,
        -1.47005123e-04,  2.71811477e-01, -2.05738037e-01,
         3.61989880e-01, -2.85510745e-01]])

In [43]:
unscaled_inputs.columns.values

array(['Reason_1', 'Reason_2', 'Reason_3', 'Reason_4', 'Month_Value',
       'Day of the Week', 'Transportation Expense', 'Distance to Work',
       'Age', 'Daily Work Load Average', 'Body Mass Index', 'Education',
       'Children', 'Pets'], dtype=object)

In [44]:
feature_name = unscaled_inputs.columns.values

In [45]:
summary_table = pd.DataFrame(data=feature_name,columns=['Feature Name'])
summary_table['Coefficient'] = np.transpose(reg.coef_)    #transpose is made to convert nd array(row) into column
summary_table

,Feature Name,Coefficient
0,Reason_1,2.800965
1,Reason_2,0.934858
2,Reason_3,3.095616
3,Reason_4,0.856587
4,Month_Value,0.166248
5,Day of the Week,-0.084370
6,Transportation Expense,0.612733
7,Distance to Work,-0.007797
8,Age,-0.165923
9,Daily Work Load Average,-0.000147


In [46]:
# in the above table intercept have to be added.. but we want intercept at the first.So we will move forward indices by 1
summary_table.index = summary_table.index + 1

In [47]:
summary_table.loc[0]= ['Intercept',reg.intercept_[0]]

In [48]:
summary_table

,Feature Name,Coefficient
1,Reason_1,2.800965
2,Reason_2,0.934858
3,Reason_3,3.095616
4,Reason_4,0.856587
5,Month_Value,0.166248
6,Day of the Week,-0.084370
7,Transportation Expense,0.612733
8,Distance to Work,-0.007797
9,Age,-0.165923
10,Daily Work Load Average,-0.000147


In [49]:
summary_table = summary_table.sort_index()

In [50]:
summary_table

,Feature Name,Coefficient
0,Intercept,-1.656109
1,Reason_1,2.800965
2,Reason_2,0.934858
3,Reason_3,3.095616
4,Reason_4,0.856587
5,Month_Value,0.166248
6,Day of the Week,-0.084370
7,Transportation Expense,0.612733
8,Distance to Work,-0.007797
9,Age,-0.165923


### Interpreting the coefficieints

In [51]:
summary_table['Odds_ratio'] = np.exp(summary_table.Coefficient)
summary_table

,Feature Name,Coefficient,Odds_ratio
0,Intercept,-1.656109,0.190880
1,Reason_1,2.800965,16.460523
2,Reason_2,0.934858,2.546851
3,Reason_3,3.095616,22.100858
4,Reason_4,0.856587,2.355110
5,Month_Value,0.166248,1.180866
6,Day of the Week,-0.084370,0.919091
7,Transportation Expense,0.612733,1.845467
8,Distance to Work,-0.007797,0.992233
9,Age,-0.165923,0.847112


We will sort the summary table as per the odds_ratio in descending order. That will give us important reasons for absenteeism at top. 
A feature is not particularly important:
- if its coefficient is around 0
- if its odd ratio is around 1

In [52]:
summary_table.sort_values('Odds_ratio',ascending= False)

,Feature Name,Coefficient,Odds_ratio
3,Reason_3,3.095616,22.100858
1,Reason_1,2.800965,16.460523
2,Reason_2,0.934858,2.546851
4,Reason_4,0.856587,2.355110
7,Transportation Expense,0.612733,1.845467
13,Children,0.361990,1.436184
11,Body Mass Index,0.271811,1.312340
5,Month_Value,0.166248,1.180866
10,Daily Work Load Average,-0.000147,0.999853
8,Distance to Work,-0.007797,0.992233


So from the above table we can see that daily work load avearge,distance to work and day of the week are not much important features for being absent/ these dont make much differnce .

In [53]:
reg.score(x_test,y_test)

0.7428571428571429

So our model is predicting the excessive absenteeism with an accuracy of 74% for the data which is not seen by the model. i.e. test data. 

Test accuracy is always LESS than train accuracy. If test accuracy is more then we may have made some mistake in model.

Testing should be done once only.If we are doing testing repeatedly with modifications of parameters then its nothing but a "TRAINING".So testing should be done only once.

## Save the model

In [54]:
# import the relevant module
import pickle

In [55]:
# pickle the model file
with open('model.pkl', 'wb') as file:
    pickle.dump(reg, file)

In [56]:
# pickle the scaler file
with open('scaler','wb') as file:
    pickle.dump(absenteeism_scaler, file)